<a href="https://colab.research.google.com/github/M-Jak/Finki/blob/main/Voved%20vo%20nauka%20na%20podatoci/custom_scraper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
from IPython.display import HTML

import warnings

requests.packages.urllib3.disable_warnings()
warnings.filterwarnings("ignore")

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
response = requests.get("https://setec.mk/index.php?route=product/category&path=10019_10020&sort=p.sort_order&order=ASC")
response

<Response [200]>

In [44]:
html = BeautifulSoup(response.text,"html.parser")
# name -> div.right -> a -> text -> strip
# price -> div.right -> span.price-old-new -> text
# price ackiska -> div.right -> span.price-new-new -> text
# shifra -> div.right -> div.shifra -> text
html.select_one('div.right').select_one("a").text.strip()

'16GB 3600MHz DDR4 CL17 DIMM XMP HyperX Predator'

In [76]:
def scrape(url="https://setec.mk/index.php?route=product/category&path=10019_10020&sort=p.sort_order&order=ASC&limit=100",path="/content/drive/MyDrive/VNP labs/AUD scraping/data"):
  response = requests.get(url)
  html = BeautifulSoup(response.text,"html.parser")
  print(response.status_code)
  produkti_html = html.find_all("div",{"class":"right"})
  print(len(produkti_html))
  products = []

  for i in range(len(produkti_html)):
    item = {}
    print(produkti_html[i])
    item['ID'] = i+1
    item['Name'] = produkti_html[i].select_one('a').text.strip()
    item['Regular price'] = produkti_html[i].select_one('span.price-old-new').text
    item['Discounted price'] = produkti_html[i].select_one('span.price-new-new').text
    item['Shifra'] = produkti_html[i].select_one('div.shifra').text.split(":")[1].strip()

    products.append(item)

  df = pd.DataFrame(products)
  url_parts = url.split("/")
  filename = path + "/results_" + url_parts[2].split(".")[0] + ".csv"
  df.to_csv(filename, index=False)

data = scrape("https://setec.mk/index.php?route=product/category&path=10066_10067&sort=p.price&order=DESC")
data

200
31
<div class="right"> <div class="name" id="mora_da_ima_prazno_mesto"><a href="https://setec.mk/index.php?route=product/product&amp;path=10066_10067&amp;product_id=56795&amp;sort=p.price&amp;order=DESC"> Apple iPhone 15 Pro Max 1TB Natural Titanium</a> </div> <div class="shifra"> Шифра: 56795 </div> <div class="lager"> <div class="nema_zaliha" style="color:#f51b1b;">Производот не е достапен</div> </div> <div class="price"> <div class="cena-bace-cena-druze"> <div class="category-price-redovna"> <span class="redovna-cena">Редовна цена: </span> <span class="price-old-new">142,990 Ден.</span> </div> <div class="category-price-akciska"> <span class="akciska-cena">КЛУБ ЦЕНА: </span> <p style="clear: both; text-align: center; margin: 2px !important;"></p> <span class="price-new-new">131,290 Ден.</span> </div> <div id="category-rata"> <span class="klub-rata">12 РАТИ: </span> <span class="klub-rata-suma"> 11,378 Ден.</span><br/> <span class="klub-rata">24 РАТИ: </span> <span class="klub-ra